In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, auto_fit_columns
from bulletin.commom import static

from hashlib import sha256
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf, normalize_ibge

import gc

pd.set_option('display.max_columns', None)

output = join("output","correcoes","tb_pacientes")

if not isdir(output):
    makedirs(output)

regionais = static.regionais.copy()
timer = Timer()

In [2]:
timer.reset()
tb = TbPacientes()
# tb.update()
timer.stop()

1.87420654296875

In [3]:
timer.reset()
casos_confirmados = CasosConfirmados()
# casos_confirmados.update()
timer.stop()

1.548377275466919

In [4]:
casost = tb.get()
casosc = casos_confirmados.get_casos()
print(f"casos casos_confirmados: {casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")

casos casos_confirmados: (824044, 24)
casos tb_pacientes: (824044, 48)


In [5]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_tb_pacientes = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')

In [6]:
idx_tb_pacientes = casost[casost['hash'].isin(casosc['hash'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_less'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_more'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_casos_confirmados = casost.loc[ set(casost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')


In [7]:
print(f"casos_confirmados_nao_tb_pacientes: {casos_confirmados_nao_tb_pacientes.shape}")
if len(casos_confirmados_nao_tb_pacientes) > 0:
    casos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'casos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_casos_confirmados: {tb_pacientes_nao_casos_confirmados.shape}")
if len(tb_pacientes_nao_casos_confirmados) > 0:
    tb_pacientes_nao_casos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_casos_confirmados.xlsx'), index=False)
if (len(casos_confirmados_nao_tb_pacientes) > 0) or (len(tb_pacientes_nao_casos_confirmados) > 0):
    raise Exception("diferentes")

casos_confirmados_nao_tb_pacientes: (0, 24)
tb_pacientes_nao_casos_confirmados: (0, 48)


In [8]:
casost['ibge_res_pr'] = casost['ibge_res_pr'].fillna('9999999')
casosc['ibge7'] = casosc['ibge7'].fillna('9999999')

In [9]:
casosc['nome_hash'] = casosc['nome'].apply(normalize_hash)
casost['nome_hash'] = casost['nome'].apply(normalize_hash)

In [10]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
print(all(casosc['idade'].values == casost['idade'].values))

False
False
False


In [11]:
timer.reset()

if False:
    sorted_casosc = pd.DataFrame(columns=casosc.columns, dtype=casosc.dtypes.values)
    sorted_casost = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)
    i = 0
    for (ibge_cc, df_ibge_cc), (ibge_tb, df_ibge_tb) in zip(casosc.groupby('ibge7'),casost.groupby('ibge_res_pr')):
        print(i,end=' ')
        i+=1
        if ibge_cc != ibge_tb:
            raise Exception(ibge_cc,ibge_tb)

        df_ibge_cc = df_ibge_cc.sort_values('nome_hash')
        df_ibge_tb = df_ibge_tb.sort_values('nome_hash')

        duplicados_ibge_cc = df_ibge_cc.loc[ df_ibge_cc.duplicated('nome_hash', keep=False) ]
        duplicados_ibge_tb = df_ibge_tb.loc[ df_ibge_tb.duplicated('nome_hash', keep=False) ]

        sorted_casosc = sorted_casosc.append(df_ibge_cc.loc[~df_ibge_cc.index.isin(duplicados_ibge_cc.index)])
        sorted_casost = sorted_casost.append(df_ibge_tb.loc[~df_ibge_tb.index.isin(duplicados_ibge_tb.index)])

        for (nome_cc, df_nome_cc), (nome_tb, df_nome_tb) in zip(duplicados_ibge_cc.groupby('nome_hash'),duplicados_ibge_tb.groupby('nome_hash')):
            if nome_cc != nome_tb:
                raise Exception(nome_cc,nome_tb)

            sorted_casosc = sorted_casosc.append(df_nome_cc.sort_values('idade'))
            sorted_casost = sorted_casost.append(df_nome_tb.sort_values('idade'))


    print(f"casos casos_confirmados: {sorted_casosc.shape}")
    print(f"casos tb_pacientes: {casost.shape}")
    print(timer.stop())
    casosc = sorted_casosc
    casost = sorted_casost

else:
    casosc['identificacao'] = casosc['identificacao'].apply(int)
    casost['identificacao'] = casost['identificacao'].apply(int)
    casosc = casosc.sort_values('identificacao').copy()
    casost = casost.sort_values('identificacao').copy()

In [12]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
print(all(casosc['idade'].values == casost['idade'].values))

False
True
False


In [13]:
all(casosc.sort_values('nome_hash')['nome_hash'].values == casost.sort_values('nome_hash')['nome_hash'].values)

True

In [14]:
obitosc = casosc.loc[casosc['obito']=='SIM']
obitost = casost.loc[casost['obito']=='SIM']

In [15]:
obitosc.loc[~obitosc['identificacao'].isin(obitost['identificacao'])]

,ordem,identificacao,ibge7,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_dt_diag,nome_hash


In [16]:
obitost.loc[~obitost['identificacao'].isin(obitosc['identificacao'])]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,15_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais1,idade_menos1,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,nome_hash


In [17]:
regionais = static.regionais.copy()
try:
    casosc = casosc_copy.copy()
except:
    casosc_copy = casosc.copy()

casosc['identificacao'] = casost['identificacao'].values.copy()

casosc['ordem'] = [ i for i in range(1,len(casosc)+1)]

casosc = casosc.rename(columns={'ibge7':'ibge_resid','nome':'paciente','dt_diag':'data_diagnostico','comunicacao':'data_comunicacao','is':'data_1o_sintomas'})

casosc['ibge_atend'] = casost['ibge_atend_pr'].apply(normalize_ibge).values.tolist()
casosc['ibge_resid'] = casosc['ibge_resid'].apply(normalize_ibge).values.tolist()

casosc.loc[casosc['rs'].isnull(),'rs'] = 99
casosc['rs'] = casosc['rs'].apply(int)

regionais = regionais.append(pd.Series({'nu_reg':99,'nm_macro':"FORA_PR",'nm_reg':'FORA_PR'}),ignore_index=True)
regionais = regionais.groupby(['nu_reg','nm_reg','nm_macro'])[['ibge']].count().reset_index()
del regionais['ibge']
# casosc['nm_macro'] = casosc.apply(lambda x: regionais.loc[regionais['nu_reg']==x['rs']],1)
# casosc['nm_reg'] = casosc.apply(lambda x: regionais.loc[regionais['nu_reg']==x['rs']],1)
casosc = pd.merge(casosc,regionais,how='left',left_on='rs',right_on='nu_reg',left_index=True,validate = 'm:1')


casosc = pd.merge(casosc,casost[['identificacao','dt_obito','dt_com_obito','status','dt_com_recuperado']],'left',on='identificacao')

casosc['caso'] = 1
casosc['recuperado'] = 0
casosc['ativo'] = 1

casosc.loc[(casosc['status']=="Recuperado"),'recuperado'] = 1
casosc.loc[(casosc['obito']=='SIM'),'obito'] = 1
casosc.loc[(casosc['obito']=='NAO'),'obito'] = 0
casosc.loc[(casosc['obito']==1) | (casosc['recuperado']==1),'ativo'] = 0

casosc.loc[(casosc['recuperado']==1) & (casosc['obito']==1 ), 'recuperado'].to_excel('casos_cura_obito.xlsx')
casosc.loc[(casosc['recuperado']==1) & (casosc['obito']==1 ), 'recuperado'] = 0

# casosc['evolucao'] = 'ATIVO'
# casosc.loc[casosc['recuperado']==1, 'evolucao'] = 'CURA'
# casosc.loc[casosc['obito']==1, 'evolucao'] = 'OBITO'

# casosc.loc[casosc['recuperado']==1, 'data_com_evolucao'] = casosc.loc[casosc['recuperado']==1, 'data_com_recuperado']
# casosc.loc[casosc['obito']==1, 'data_com_evolucao'] = casosc.loc[casosc['obito']==1, 'data_com_obito']

# casosc.loc[casosc['obito']==1, 'data_evolucao'] = casosc.loc[casosc['obito']==1, 'data_obito']

casosc.loc[casosc['obito']==1,'evolucao'] = 'OBITO'
casosc.loc[casosc['recuperado']==1,'evolucao'] = 'RECUPERADO'
casosc.loc[casosc['ativo']==1,'evolucao'] = 'ATIVO'

casosc = casosc.rename(columns={'dt_com_obito':'data_com_evolucao','data_obito':'data_evolucao'})

casosc = casosc[['nm_macro','ordem', 'ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'rs','nm_reg', 'mun_resid', 'mun_atend', 'laboratorio', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas', 'evolucao','data_evolucao', 'data_com_evolucao']].sort_values('ordem')

print(casosc.columns)
casosc

Index(['nm_macro', 'ordem', 'ibge_resid', 'ibge_atend', 'paciente', 'sexo',
       'idade', 'rs', 'nm_reg', 'mun_resid', 'mun_atend', 'laboratorio',
       'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas', 'evolucao',
       'data_evolucao', 'data_com_evolucao'],
      dtype='object')


,nm_macro,ordem,ibge_resid,ibge_atend,paciente,sexo,idade,rs,nm_reg,mun_resid,mun_atend,laboratorio,data_diagnostico,data_comunicacao,data_1o_sintomas,evolucao,data_evolucao,data_com_evolucao
0,OESTE,1,410830,410830,ABBAS MOHAMAD MESSELMANI,M,41,9,FOZ DO IGUACU,FOZ DO IGUACU,FOZ DO IGUACU,LACEN,2020-04-17,2020-04-18,2020-04-11,RECUPERADO,NaT,NaT
1,NOROESTE,2,411710,411710,ABDIEL DE LIMA GONCALVES,M,44,14,PARANAVAI,NOVA LONDRINA,NOVA LONDRINA,IBMP,2020-07-03,2020-07-04,2020-06-22,RECUPERADO,NaT,NaT
2,OESTE,3,412770,412770,ABDOU INDIAYE,M,31,20,TOLEDO,TOLEDO,TOLEDO,IBMP,2020-06-03,2020-06-04,2020-05-29,RECUPERADO,NaT,NaT
3,LESTE,4,411200,411200,ABEDIEL DA SILVA,M,38,3,PONTA GROSSA,JAGUARIAIVA,JAGUARIAIVA,COVID-19 IGG/IGM ECO TESTE,2020-06-05,2020-06-06,NaT,RECUPERADO,NaT,NaT
4,LESTE,5,412550,412550,ABEGAIL DOMINGUES,F,59,2,METROPOLITANA,SAO JOSE DOS PINHAIS,SAO JOSE DOS PINHAIS,IBMP,2020-06-16,2020-06-17,2020-06-09,RECUPERADO,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824039,OESTE,824040,412770,412770,ZULEICA DE LOURDES PELIZARO FRUGERIO,F,76,20,TOLEDO,TOLEDO,TOLEDO,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2021-03-15,2021-03-25,NaT,ATIVO,NaT,NaT
824040,LESTE,824041,410690,410690,ZULEMIA ANNA DO VALLE RIBEIRO,F,85,2,METROPOLITANA,CURITIBA,CURITIBA,COVID-19 BIOLOGIA MOLECULAR,2021-03-19,2021-03-25,2021-03-15,ATIVO,NaT,NaT
824041,NORTE,824042,411370,411370,ZULENE DE OLIVEIRA BEVILACQUA,F,59,17,LONDRINA,LONDRINA,LONDRINA,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2021-03-24,2021-03-25,2021-03-24,ATIVO,NaT,NaT
824042,NOROESTE,824043,412625,412625,ZULMIRA CONCEICAO FERNANDES ORLANDO,F,71,15,MARINGA,SARANDI,SARANDI,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2021-03-11,2021-03-25,NaT,ATIVO,NaT,NaT


In [18]:
casosc.groupby('evolucao')[['ordem']].count()

,ordem
evolucao,
ATIVO,228802
OBITO,15750
RECUPERADO,579492


In [35]:
totais_macro = casosc.groupby(['nm_macro','evolucao'])[['ordem']].count()
totais_macro = totais_macro.unstack()
totais_macro['TOTAL_CASOS'] = totais_macro.sum(1)
totais_macro
# fora_pr = totais_macro.iloc[0]
totais_macro.append(pd.Series(totais_macro.sum(0),name='TOTAL GERAL'))

ordem                   TOTAL_CASOS
evolucao      ATIVO  OBITO RECUPERADO            
nm_macro                                         
FORA_PR        3267    117       1950        5334
LESTE        105112   7915     268761      381788
NOROESTE      34883   2162      92228      129273
NORTE         59045   2685      70334      132064
OESTE         26495   2871     146219      175585
TOTAL GERAL  228802  15750     579492      824044

In [36]:
totais_rs = casosc.groupby(['rs','nm_reg','evolucao'])[['ordem']].count()
totais_rs = totais_rs.unstack().droplevel(0,1)
totais_rs['TOTAL_CASOS'] = totais_rs.sum(1)
totais_rs.append(pd.Series(totais_rs.sum(0),name=('TOTAL','GERAL')))

,evolucao,ATIVO,OBITO,RECUPERADO,TOTAL_CASOS
rs,nm_reg,,,,
1,PARANAGUA,10425,510,14195,25130
2,METROPOLITANA,60893,5748,193278,259919
3,PONTA GROSSA,15118,831,28813,44762
4,IRATI,1336,139,6842,8317
5,GUARAPUAVA,8487,319,13202,22008
6,UNIAO DA VITORIA,1504,98,4547,6149
7,PATO BRANCO,3297,393,17280,20970
8,FRANCISCO BELTRAO,6793,415,21646,28854
9,FOZ DO IGUACU,9498,760,35039,45297


In [23]:
aux = casosc.copy()
aux.loc[aux['rs']==99,'mun_resid'] = 'Fora do Paraná'
totais_mun = aux.groupby(['rs','nm_macro','ibge_resid','mun_resid','evolucao'])[['ordem']].count()
totais_mun = totais_mun.unstack()
totais_mun['TOTAL_CASOS'] = totais_mun.sum(1)
totais_mun.append(pd.Series(totais_mun.sum(0),name='TOTAL_GERAL'))
totais_mun = totais_mun.fillna(0).astype(int)
totais_mun

ordem                  TOTAL_CASOS
evolucao                                    ATIVO OBITO RECUPERADO            
rs nm_macro ibge_resid mun_resid                                              
1  LESTE    410120     ANTONINA               140    44       1514        1698
            410950     GUARAQUECABA            42     8        271         321
            410960     GUARATUBA              913   106       2635        3654
            411570     MATINHOS               185    57       1716        1958
            411620     MORRETES               254    31       1177        1462
...                                           ...   ...        ...         ...
22 NORTE    412265     ROSARIO DO IVAI         32     2        178         212
            412385     SANTA MARIA DO OESTE   116    12        299         427
            412500     SAO JOAO DO IVAI        26     9        523         558
99 FORA_PR  410480     Fora do Paraná           0     1          0           1
            999999     Fora do Paraná        3267   116       1950        5333

[403 rows x 4 columns]

In [45]:
casosc.loc[(casosc['nm_macro']=='FORA_PR') & (casosc['ibge_resid']==410480)]

,nm_macro,ordem,ibge_resid,ibge_atend,paciente,sexo,idade,rs,nm_reg,mun_resid,mun_atend,laboratorio,data_diagnostico,data_comunicacao,data_1o_sintomas,evolucao,data_evolucao,data_com_evolucao
626814,FORA_PR,626815,410480,410480,ONILSON CANDIDO DE JESUS,M,63,99,FORA_PR,CASCAVEL,CASCAVEL,VIRUS RESPIRATORIO,2021-02-22,2021-02-24,2021-02-11,OBITO,2021-03-02,2021-03-14


In [24]:
exit
writer = pd.ExcelWriter(join(output,"casos_confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

for macro, macro_df in casosc.groupby('nm_macro'):
    macro_df['ordem'] = [ i for i in range(1,len(macro_df)+1)]
    macro_df['cross_idx'] = [ f"{macro}{i}" for i in range(1,len(macro_df)+1)]
    macro_df = macro_df[['ordem','cross_idx','ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'rs', 'mun_resid', 'mun_atend', 'laboratorio', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas', 'evolucao', 'data_evolucao', 'data_com_evolucao']].sort_values('ordem')
    del macro_df['ordem']
    macro_df.to_excel(writer,macro,index=None)
    worksheet = writer.sheets[macro]
    auto_fit_columns(worksheet,macro_df)

writer.save()

SystemExit: 

In [ ]:
exit()
obitosc = casosc.loc[casosc['evolucao']=='OBITO']

In [ ]:
writer = pd.ExcelWriter(join(output,"obitos_confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

for macro, df in obitosc.groupby('macro'):
# casosc.to_excel(writer,'geral',index=None)
# worksheet = writer.sheets[f"geral"]
# auto_fit_columns(worksheet,casosc)

writer.save()
writer.close()